In [1]:
import pandas as pd
import numpy as np

## 思维方式的转变
以向量集合数据作为整体对象进行操作，以往针对单个对象的操作符同样适合集合对象，如加减乘除，既可以对单个数，也可以对一个向量或矩阵，GPU非常擅长矩阵数据的处理，把数据组织成有序向量可以极大提高处理速度，因此每个步骤优先考虑是否能转换成向量或矩阵操作，尽量避免循环遍历，分支判断，pandas提供了这样的设施

## 基本数据结构
* Series: 1维数组，支持两类索引(Index)：1. 位置(position: int)，2. 标签(label: object|string)
* DataFrame: 2维数据表格，由Series构成列(Columns), 并共用Series的索引(Index), 所以表格行列称为(index,columns)或(rows,columns), 也可称为第0纬和第1维, 或第0轴和第1轴，在指定数据方向的时候，这几种方式是等价的, 即 axis=0|index|rows 指定数据垂直方向, axis=1|columns 指定数据水平方向

In [2]:
s = pd.Series([1,2,3,4,5])
s

0    1
1    2
2    3
3    4
4    5
dtype: int64

In [3]:
s1 = pd.Series(list('abcde'))
s1

0    a
1    b
2    c
3    d
4    e
dtype: object

In [4]:
df = pd.DataFrame({'col0':s, 'col1':s1})
df

,col0,col1
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e


### DataFrame 相关信息

In [5]:
# 数据的形状为 10x5 = len(df.index) x len(df.columns)
df.shape

(5, 2)

In [6]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [7]:
df.columns

Index(['col0', 'col1'], dtype='object')

In [8]:
# 统计每列非空个数
df.loc[3,'b'] = np.nan
df.count(axis=0)

col0    5
col1    5
b       0
dtype: int64

### 指定坐标轴操作

In [9]:
df = pd.DataFrame(np.random.randint(0,10,size=(5,3)))
df

,0,1,2
0,9,9,9
1,5,0,6
2,1,0,9
3,3,5,8
4,8,4,3


In [10]:
# 行方向求和
df.sum(axis=0)

0    26
1    18
2    35
dtype: int64

In [11]:
# 列方向求和
df.sum(axis=1)

0    27
1    11
2    10
3    16
4    15
dtype: int64

In [12]:
# 丢弃0轴上的 1,3 坐标点
df.drop([1,3], axis=0)

,0,1,2
0,9,9,9
2,1,0,9
4,8,4,3


In [13]:
# 丢弃1轴上的 2 坐标点
df.drop([2], axis=1)

,0,1
0,9,9
1,5,0
2,1,0
3,3,5
4,8,4


### 固定坐标轴操作
```df[key]```中的key为列轴坐标，但当key为bool Series类型时为行掩码，```df.loc[x,y]```中的x, y分别为行轴，列轴坐标

In [14]:
df = pd.DataFrame(np.random.randint(0,10,size=(5,3)), columns=list('abc'))
df

,a,b,c
0,0,8,9
1,7,8,7
2,1,9,8
3,4,2,4
4,1,3,4


In [15]:
# 1 轴(columns)上的操作, 通过key='a'定位列数据
df['a']

0    0
1    7
2    1
3    4
4    1
Name: a, dtype: int64

In [16]:
# 0 轴(index)上的操作，通过key=3定位行数据, 行数据也是Series，一般会统一转成object类型，因为不同列一般类型不同
df.loc[3]

a    4
b    2
c    4
Name: 3, dtype: int64

In [17]:
# 0,1 两个坐标轴定位赋值
df.loc[3,'a'] = 30
df

,a,b,c
0,0,8,9
1,7,8,7
2,1,9,8
3,30,2,4
4,1,3,4


### 重排索引/对齐索引
对向量集合进行运算的时候，需要对齐索引，这样才能进行有意义的运算，实际上pandas默认会先对齐索引，再进行运算

In [171]:
s1 = pd.Series(data=[1,2,3], index=list('abc'))
s1

a    1
b    2
c    3
dtype: int64

In [172]:
s2 = pd.Series(data=[5,6,7], index=list('bcd'))
s2

b    5
c    6
d    7
dtype: int64

In [173]:
# 先对齐索引再相加，对不齐的用nan填充
s1+s2

a    NaN
b    7.0
c    9.0
d    NaN
dtype: float64

In [174]:
s1*s2

a     NaN
b    10.0
c    18.0
d     NaN
dtype: float64

In [177]:
# 手动对齐再运算
idx = s1.index.union(s2.index)
idx

Index(['a', 'b', 'c', 'd'], dtype='object')

In [178]:
s1.reindex(idx)

a    1.0
b    2.0
c    3.0
d    NaN
dtype: float64

In [179]:
s2.reindex(idx)

a    NaN
b    5.0
c    6.0
d    7.0
dtype: float64

In [181]:
# 对齐索引后再运算
s1.reindex(idx) + s2.reindex(idx)

a    NaN
b    7.0
c    9.0
d    NaN
dtype: float64

In [182]:
# 以 s1 为准
s1 + s2.reindex(s1.index)

a    NaN
b    7.0
c    9.0
dtype: float64

### 数据类型 dtypes
每个Series都有一个dtype，表示序列的类型，一般是一个具体的类型，混合类型用object，如果取一行数据，一般用混合的object类型，常用类型如下：

* int
* float
* string
* bool
* object

In [18]:
# 如果不指定类型，会自动推断类型，string 会推断成object
s = pd.Series(['hello','world','nice','to','meet','pandas'])
s

0     hello
1     world
2      nice
3        to
4      meet
5    pandas
dtype: object

In [19]:
# 最好指定具体类型，如下数据在读取csv的时候容易推断成 float 类型， 给操作带来不便
s = pd.Series(['13122334455','13888888884',np.nan], dtype='string')
s

0    13122334455
1    13888888884
2           <NA>
dtype: string

## 更多数据定位选取与赋值
* by position
* by index/label
* by boolean index

In [20]:
df = pd.DataFrame(np.random.randint(0,100,size=(10,5)), columns=list('abcde'))
df

,a,b,c,d,e
0,91,2,8,36,15
1,33,26,22,0,20
2,24,21,83,51,48
3,34,0,55,27,0
4,63,97,2,54,58
5,54,65,88,40,61
6,79,45,56,80,94
7,66,38,96,26,70
8,35,57,46,14,49
9,3,24,82,83,31


### 通过位置选取数据 (数组下标)

In [21]:
# 选择第3，4行数据
df.iloc[[3,4]]

,a,b,c,d,e
3,34,0,55,27,0
4,63,97,2,54,58


In [22]:
# 选择第3,4,5行与第2，3列数据
df.iloc[[3,4,5],[2,3]]

,c,d
3,55,27
4,2,54
5,88,40


### 通过索引选取数据

In [23]:
# 通过列索引选取数据
df['b']

0     2
1    26
2    21
3     0
4    97
5    65
6    45
7    38
8    57
9    24
Name: b, dtype: int64

In [24]:
# 通过行索引和列索引选取数据
df.loc[[1,2,5], ['a','b','d']]

,a,b,d
1,33,26,0
2,24,21,51
5,54,65,40


In [25]:
# 通过 bool Series index 选择数据, Series长度要和df行数相同，注意此时key=s不再是列坐标，而是行坐标，与loc效果相同
s = pd.Series([True,True,False,False,True,False,False,False,False,False])
df[s]

,a,b,c,d,e
0,91,2,8,36,15
1,33,26,22,0,20
4,63,97,2,54,58


In [26]:
# 与上面效果相同
df.loc[s]

,a,b,c,d,e
0,91,2,8,36,15
1,33,26,22,0,20
4,63,97,2,54,58


In [27]:
# bool Series index 的威力在于组合各种条件筛选数据，如下面的条件
mask = (df['a'] > 30) & (df['b'] % 2 == 0)
mask

0     True
1     True
2    False
3     True
4    False
5    False
6    False
7     True
8    False
9    False
dtype: bool

In [28]:
# 类似于数据的行掩码，只选取为True的行
df[mask]

,a,b,c,d,e
0,91,2,8,36,15
1,33,26,22,0,20
3,34,0,55,27,0
7,66,38,96,26,70


### 赋值
如果对数据进行链式索引再赋值，如
 ```df[col][row] = value```，底层操作实际上是```df.__getitem__(col).__setitem__(row, value)```, 第一级的索引返回的可能是self或copy，后面赋值不可预知，完成此操作应该使用loc进行一次性索引定位，```df.loc[row_indexer, column_indexer] = value```

In [29]:
# 使用chain indexing 赋值会有SettingWithCopyWarning
pd.set_option('mode.chained_assignment','warn')
df[mask][1] = -1
df

/var/folders/x9/mnh091hx4378zn3qc931vgtc0000gn/T/ipykernel_68714/1034507843.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[mask][1] = -1


,a,b,c,d,e
0,91,2,8,36,15
1,33,26,22,0,20
2,24,21,83,51,48
3,34,0,55,27,0
4,63,97,2,54,58
5,54,65,88,40,61
6,79,45,56,80,94
7,66,38,96,26,70
8,35,57,46,14,49
9,3,24,82,83,31


In [30]:
# 使用loc赋值，不要使用chain-indexing
df.loc[mask, 'a'] = -1
df

,a,b,c,d,e
0,-1,2,8,36,15
1,-1,26,22,0,20
2,24,21,83,51,48
3,-1,0,55,27,0
4,63,97,2,54,58
5,54,65,88,40,61
6,79,45,56,80,94
7,-1,38,96,26,70
8,35,57,46,14,49
9,3,24,82,83,31


## 遍历行

In [31]:
data = np.random.choice(list('abcdefghijklmnopqrstuvwxyz'), size=(5,3))
df = pd.DataFrame(data=data, columns=list('abc'))
df

,a,b,c
0,t,e,c
1,l,e,t
2,n,z,j
3,r,t,d
4,p,f,z


In [32]:
# 使用iterrows, 会把一行用Series表示，会转化到同一种类型，如int可能转化成float，多种类型转成object
for idx, row in df.iterrows():
    vals = [f'{c} = {row[c]}' for c in df.columns]
    print(idx,':',vals)

0 : ['a = t', 'b = e', 'c = c']
1 : ['a = l', 'b = e', 'c = t']
2 : ['a = n', 'b = z', 'c = j']
3 : ['a = r', 'b = t', 'c = d']
4 : ['a = p', 'b = f', 'c = z']


In [33]:
# 相比iterrows, itertuples保持类型，速度更快
for row in df.itertuples():
    vals = [f'{c} = {getattr(row,c)}' for c in df.columns]
    print(row.Index,':',vals)

0 : ['a = t', 'b = e', 'c = c']
1 : ['a = l', 'b = e', 'c = t']
2 : ['a = n', 'b = z', 'c = j']
3 : ['a = r', 'b = t', 'c = d']
4 : ['a = p', 'b = f', 'c = z']


## string 类型
.str 属性支持常用的字符串操作

In [132]:
s = pd.Series(['hello', 'world'])
s

0    hello
1    world
dtype: object

In [133]:
s.str.count('l')

0    2
1    1
dtype: int64

In [141]:
# 手机号序列
phones = pd.Series(['321321','12345678901','13144556698'])
phones

0         321321
1    12345678901
2    13144556698
dtype: object

In [135]:
# 正则表达式匹配
phones.str.match(r'^[1][3456789][0-9]{9}$')

0    False
1    False
2     True
dtype: bool

In [142]:
# 提取分子/分母
s = pd.Series(['3/5','6/7','80/100'])
s.str.extract(r'(\d)/(\d)')

,0,1
0,3,5
1,6,7
2,0,1


## 时间/时间序列
pandas 处理时间序列非常方便，

In [115]:
pd.to_datetime(['2022-05-01 10:00:00','5/2/2022','5/3/22']) + pd.DateOffset(days=3)

DatetimeIndex(['2022-05-04 10:00:00', '2022-05-05 00:00:00',
               '2022-05-06 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [117]:
dti = pd.date_range("2022-01-01", periods=3, freq="H")
dti

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 01:00:00',
               '2022-01-01 02:00:00'],
              dtype='datetime64[ns]', freq='H')

In [120]:
df = pd.DataFrame(pd.date_range("2022-01-01", periods=3, freq="D"))
df

,0
0,2022-01-01
1,2022-01-02
2,2022-01-03


In [127]:
# .dt 支持时间相关操作, month, day, minute, second等
df[0].dt.year

0    2022
1    2022
2    2022
Name: 0, dtype: int64

In [131]:
# 时间差
df[0] - pd.DateOffset(months = 12, days = 3)

0   2020-12-29
1   2020-12-30
2   2020-12-31
Name: 0, dtype: datetime64[ns]

## GroupBy: 分组 --> 组数据处理 --> 合并
分组与聚合函数与sql类似，但GroupBy后面还可以跟transform函数，此函数不会聚合数据，而是返回和分组结构一样的数据

In [143]:
df = pd.DataFrame(
    {
        "animal": "cat dog cat fish dog cat cat".split(),
        "size": list("SSMMMLL"),
        "weight": [8, 10, 11, 1, 20, 12, 12],
        "adult": [False] * 5 + [True] * 2,
    }
)
df

,animal,size,weight,adult
0,cat,S,8,False
1,dog,S,10,False
2,cat,M,11,False
3,fish,M,1,False
4,dog,M,20,False
5,cat,L,12,True
6,cat,L,12,True


In [165]:
# 按animal分组计算weight和
df.groupby('animal')['weight'].sum()

animal
cat     43
dog     30
fish     1
Name: weight, dtype: int64

### GroupBy 配合排序
GroupBy 会保持原顺序不变，这个特性可以实现分组后对组内排序一样

In [158]:
# 按 animal 和 weight 排序
df = df.sort_values(['animal','weight'])
df

,animal,size,weight,adult
0,cat,S,8,False
2,cat,M,11,False
5,cat,L,12,True
6,cat,L,12,True
1,dog,S,10,False
4,dog,M,20,False
3,fish,M,1,False


In [164]:
# 按animal分组，选择一列，生成相同长度的自然增长序列作为排名（rank), 注意到索引顺序与上面相同
df['rank'] = df.groupby('animal')['weight'].transform(lambda s: list(range(len(s)))) + 1
df

,animal,size,weight,adult,rank
0,cat,S,8,False,1
2,cat,M,11,False,2
5,cat,L,12,True,3
6,cat,L,12,True,4
1,dog,S,10,False,1
4,dog,M,20,False,2
3,fish,M,1,False,1


### argmax 与 idxmax
类似的还有 argmin, idxmin
* argmax 是获取最大值对应的position，不支持分组后调用
* idxmax 是获取最大值对应的index，支持分组后调用

In [166]:
# 返回的是位置，不是索引
df['weight'].argmax()

5

In [167]:
# 返回的是索引值
df['weight'].idxmax()

4

In [170]:
# 获取每个分组中的最大weight的记录，比sql实现简单一些
df.loc[df.groupby('animal')['weight'].idxmax()]

,animal,size,weight,adult,rank
5,cat,L,12,True,3
4,dog,M,20,False,2
3,fish,M,1,False,1


## 层级索引 和 reshape
pandas只有2维表结构，如何支持多维数据呢？通过多级索引(MutiIndex)可以实现。理论上，任意维度的数据都可以压缩成1维数据，但2维数据操作是最方便的，就像我们使用的电脑界面。高纬到2维压缩形象的讲就是把嵌套的数据铺平(flatten), 维度信息堆叠到一起存到层级索引，因此索引也有了level，具体到DataFrame，因为是二维，可以有两个层级索引。

In [34]:
index = pd.MultiIndex.from_product([np.array(range(2)),np.array(range(3))],names=['r1','r2'])
index

MultiIndex([(0, 0),
            (0, 1),
            (0, 2),
            (1, 0),
            (1, 1),
            (1, 2)],
           names=['r1', 'r2'])

In [50]:
# 生成 shape=(2x3)x4 的数据
data = np.random.choice(list('abcde'), size=(len(index),4))
df = pd.DataFrame(data=data, index=index, columns=['c1','c2','c3','c4'])
df

c1 c2 c3 c4
r1 r2            
0  0   c  b  b  a
   1   e  c  c  d
   2   c  d  b  a
1  0   a  b  b  c
   1   c  d  a  e
   2   c  c  d  d

In [52]:
# 行转列,数据变矮变宽, shape=2x(3x4)
df = df.unstack()
df

c1       c2       c3       c4      
r2  0  1  2  0  1  2  0  1  2  0  1  2
r1                                    
0   c  e  c  b  c  d  b  c  b  a  d  a
1   a  c  c  b  d  c  b  a  d  c  e  d

In [54]:
df.columns

MultiIndex([('c1', 0),
            ('c1', 1),
            ('c1', 2),
            ('c2', 0),
            ('c2', 1),
            ('c2', 2),
            ('c3', 0),
            ('c3', 1),
            ('c3', 2),
            ('c4', 0),
            ('c4', 1),
            ('c4', 2)],
           names=[None, 'r2'])

In [57]:
# 列转行，数据变高变窄 shape=(2x3)x4
df = df.stack()
df

c1 c2 c3 c4
r1 r2            
0  0   c  b  b  a
   1   e  c  c  d
   2   c  d  b  a
1  0   a  b  b  c
   1   c  d  a  e
   2   c  c  d  d

In [85]:
# 数据透视表
data = {
    'id':[1001,1001,1001,1002,1002,1002],
    'variable':['姓名','体重','分数']*2,
    'value':['xiao',77,85,'liu',88,70]
}
df = pd.DataFrame(data)
df

,id,variable,value
0,1001,姓名,xiao
1,1001,体重,77
2,1001,分数,85
3,1002,姓名,liu
4,1002,体重,88
5,1002,分数,70


In [81]:
# 数据透视表，按某些数据列重新组织数据，是数据的一种reshape
df.pivot(index='id', values='value', columns='variable')

variable,体重,分数,姓名
id,,,
1001,77,85,xiao
1002,88,70,liu


In [108]:
# pivot的效果可由如下操作同样实现
df.set_index(['id','variable']).unstack()

value          
variable    体重  分数    姓名
id                      
1001        77  85  xiao
1002        88  70   liu